In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

In [ ]:
'''
os.mkdir("../input/kerasmodels")
model_names = ('ResNet50', 'ResNet50-dropout', 'InceptionV3', 'VGG16', 'EfficientNetB7', 'SimpleCNN', 'Xception', 'VGG19','InceptionResNetV2','ResNet101','ResNet101V2','ResNet152','ResNet152V2','ResNet50V2','DenseNet121','MobileNet','MobileNetV2')
model_paths = (os.path.join(name, f'{name}-log.csv') for name in model_names)
'''

In [ ]:
dir = '../input/kerasmodels/'
model_names = ('ResNet50', 'ResNet50-dropout', 'InceptionV3', 'VGG16', 'EfficientNetB7', 'SimpleCNN', 'Xception', 'VGG19','InceptionResNetV2','ResNet101','ResNet101V2','ResNet152','ResNet152V2','ResNet50V2','DenseNet121','MobileNet','MobileNetV2','DenseNet169','DenseNet201')

model_paths = list()
for name in model_names:
    s = dir+name+"/"+name+"-log.csv"
    model_paths.append(s)
model_paths = tuple(model_paths)

In [ ]:
print(model_paths)

In [ ]:
def collect_logs(model_names: tuple, model_paths: tuple, df: pd.DataFrame = None, sep:str = ';') -> pd.DataFrame:
    for name, path in zip(model_names, model_paths):

            if df is None:
                df = pd.read_csv(path, sep=sep)
                #print("in "+path)
                df['model'] = name
                df.set_index(['model', 'epoch'], inplace=True)
                continue
                
            log = pd.read_csv(path, sep=sep)
            log['model'] = name
            log.set_index(['model', 'epoch'], inplace=True)
            #print("out "+path)
            df = pd.concat([df, log])

    return df

In [ ]:
df = collect_logs(model_names, model_paths)

In [ ]:
df.head(100)

In [ ]:
df.shape

In [ ]:
# training accuracy
tr_acc = df.groupby('model').accuracy.max()
tr_loss = df.groupby('model').loss.min()
val_loss = df.groupby('model').val_loss.min()

In [ ]:
print('Final training accuracy:')
tr_acc

In [ ]:
print('Final validation accuracy:')
val_acc = df.groupby('model').val_accuracy.max()
val_acc

In [ ]:
ax = sns.barplot(x=val_acc.index, y=val_acc)
fig = plt.gcf()
fig.set_size_inches(30, 10)
#plt.bar_label(ax.containers[0], size=16,label_type='center')
plt.title('Final validation accuracy')
plt.show()

In [ ]:
sns.lineplot(data=df, x='epoch', y='val_accuracy', hue='model')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.title('Validation accuracy over time')
plt.show()

In [ ]:
best_val_list = list()
best_val_loss = list()
best_tra_list = list()
best_tra_loss = list()
for i in range(len(val_acc)):
    if(val_acc[i]>0.995):
        best_val_list.append(val_acc.index[i])
        best_val_loss.append(val_loss.index[i])
        best_tra_list.append(tr_acc.index[i])
        best_tra_loss.append(tr_loss.index[i])

In [ ]:
#best_list = ('ResNet50', 'ResNet50-dropout', 'VGG16', 'EfficientNetB7','VGG19')
best_val_df = df.query('model in @best_val_list')

In [ ]:
best_valloss_df = df.query('model in @best_val_loss')

In [ ]:
best_tra_df = df.query('model in @best_tra_list')

In [ ]:
best_traloss_df = df.query('model in @best_tra_loss')

In [ ]:
sns.lineplot(data=best_val_df, x='epoch', y='val_accuracy', hue='model')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.title('Best Validation Accuarcies of models')
plt.show()

In [ ]:
sns.lineplot(data=best_valloss_df, x='epoch', y='val_loss', hue='model')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.title('Best Validation losses of models')
plt.show()

In [ ]:
sns.lineplot(data=best_tra_df, x='epoch', y='accuracy', hue='model')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.title('Best Training Accuarcies of models')
plt.show()

In [ ]:
sns.lineplot(data=best_traloss_df, x='epoch', y='loss', hue='model')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.title('Best Training losses of models')
plt.show()

# From the above graphs it becomes clear that ResNet101 and EfficentNetB7 are the only ones with very stable curves both in training and validation for losses and accuracies across epochs respectievely

In [ ]:
#EfficientNetB7

for i in range(len(model_names)):
    if(model_names[i]=='EfficientNetB7'):
        path = model_paths[i]
        break;
#print(path)
df2 = pd.read_csv(path, sep=';')

#plot
plt.figure(figsize = (6,4))
plt.plot(df2['epoch'],df2['accuracy'],color='blue',label='Training')
plt.plot(df2['epoch'],df2['val_accuracy'],color='orange',label='Validation')
plt.title('Training and Validation Accuracy vs Epochs (EfficientNetB7)')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc="lower right")
plt.show()


plt.figure(figsize = (6,4))
plt.plot(df2['epoch'],df2['loss'],color='blue',label='Training')
plt.plot(df2['epoch'],df2['val_loss'],color='orange',label='Validation')
plt.title('Training and Validation Loss vs Epochs (EfficientNetB7)')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc="upper right")
plt.show()

In [ ]:
#ResNet101

for i in range(len(model_names)):
    if(model_names[i]=='ResNet101'):
        path = model_paths[i]
        break;
#print(path)
df2 = pd.read_csv(path, sep=';')

#plot
plt.figure(figsize = (6,4))
plt.plot(df2['epoch'],df2['accuracy'],color='blue',label='Training')
plt.plot(df2['epoch'],df2['val_accuracy'],color='orange',label='Validation')
plt.title('Training and Validation Accuracy vs Epochs (ResNet101)')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc="lower right")
plt.show()

plt.figure(figsize = (6,4))
plt.plot(df2['epoch'],df2['loss'],color='blue',label='Training')
plt.plot(df2['epoch'],df2['val_loss'],color='orange',label='Validation')
plt.title('Training and Validation Loss vs Epochs (ResNet101)')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc="upper right")
plt.show()

In [ ]:
#ResNet152

for i in range(len(model_names)):
    if(model_names[i]=='ResNet152'):
        path = model_paths[i]
        break;
#print(path)
df2 = pd.read_csv(path, sep=';')

#plot
plt.figure(figsize = (6,4))
plt.plot(df2['epoch'],df2['accuracy'],color='blue',label='Training')
plt.plot(df2['epoch'],df2['val_accuracy'],color='orange',label='Validation')
plt.title('Training and Validation Accuracy vs Epochs (ResNet152)')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc="lower right")
plt.show()


plt.figure(figsize = (6,4))
plt.plot(df2['epoch'],df2['loss'],color='blue',label='Training')
plt.plot(df2['epoch'],df2['val_loss'],color='orange',label='Validation')
plt.title('Training and Validation Loss vs Epochs (ResNet152)')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc="upper right")
plt.show()

In [ ]:
#ResNet50

for i in range(len(model_names)):
    if(model_names[i]=='ResNet50'):
        path = model_paths[i]
        break;
#print(path)
df2 = pd.read_csv(path, sep=';')

#plot
plt.figure(figsize = (6,4))
plt.plot(df2['epoch'],df2['accuracy'],color='blue',label='Training')
plt.plot(df2['epoch'],df2['val_accuracy'],color='orange',label='Validation')
plt.title('Training and Validation Accuracy vs Epochs (ResNet50)')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc="lower right")
plt.show()


plt.figure(figsize = (6,4))
plt.plot(df2['epoch'],df2['loss'],color='blue',label='Training')
plt.plot(df2['epoch'],df2['val_loss'],color='orange',label='Validation')
plt.title('Training and Validation Loss vs Epochs (ResNet50)')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#VGG16

for i in range(len(model_names)):
    if(model_names[i]=='VGG16'):
        path = model_paths[i]
        break;
#print(path)
df2 = pd.read_csv(path, sep=';')

#plot
plt.figure(figsize = (6,4))
plt.plot(df2['epoch'],df2['accuracy'],color='blue',label='Training')
plt.plot(df2['epoch'],df2['val_accuracy'],color='orange',label='Validation')
plt.title('Training and Validation Accuracy vs Epochs (VGG16)')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc="lower right")
plt.show()


plt.figure(figsize = (6,4))
plt.plot(df2['epoch'],df2['loss'],color='blue',label='Training')
plt.plot(df2['epoch'],df2['val_loss'],color='orange',label='Validation')
plt.title('Training and Validation Loss vs Epochs (VGG16)')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc="upper right")
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
all_models = list()

filename = '../input/finalized-model2/EfficientNetB7finalized_model.h5'
model = load_model(filename)
all_models.append(model)

filename = '../input/finalized-model2/ResNet152finalized_model.h5'
model = load_model(filename)
all_models.append(model)

filename = '../input/finalized-model2/ResNet50finalized_model.h5'
model = load_model(filename)
all_models.append(model)

filename = '../input/finalized-model2/VGG16finalized_model.h5'
model = load_model(filename)
all_models.append(model)


all_models.append(model)

In [ ]:
members = all_models

In [ ]:
print(len(members))

In [ ]:
input_data = pd.read_csv('../input/lungcancer-csv/test.csv')
input_data.head()

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(input_data, test_size=0.2)
print(train.shape)
print(test.shape)

In [ ]:
trainX, testX = list(train['Path']),list(test['Path'])
trainy, testy = list(train['Class']),list(test['Class'])

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import h5py
from keras import models


def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(256, 256,3))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    #img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()
  
    return img_tensor

In [ ]:
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(member,inputX):
    three = []
    #three_k = []
    stackX = []
    for i in range(len(inputX)):
        new_image = load_image(inputX[i])
        yhat = member.predict(new_image, verbose=0)
        three.append(yhat[0])
    
    stackX.append(three)
            
    # flatten predictions to [rows, members x probabilities]
    stackX = np.array(stackX)
    #print(stackX.shape[0])
    #print(stackX.shape[1])
    #print(stackX.shape[2])
    print(stackX)
    print(len(stackX))
    print(stackX.shape)
    #stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

In [ ]:
m1=stacked_dataset(members[0], testX)
t = m1

In [ ]:
'''
file = open("file1.txt", "w+")
 
# Saving the array in a text file
content = str(t)
file.write(content)
file.close()
'''

In [ ]:
m1 = t

In [ ]:
m1 = m1[0]
m1

In [ ]:
print(type(m1))

In [ ]:
print(type(m1[0]))

In [ ]:
out = list()
for i in range(len(m1)):
    a = list(m1[i])
    k = a.index(min(a))
    if (k==0):
        out.append(-1)
    elif (k==1):
        out.append(0)
    elif (k==2):
        out.append(1)

In [ ]:
print(len(out))
print(len(testy))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(testy,out)

In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(out)):
    if(testy[i]==out[i]):
        TP=TP+1
    elif()

In [ ]:
conf_matrix = confusion_matrix(y_true=testy, y_pred=out)
#
# Print the confusion matrix using Matplotlib
#
fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
# Finding precision and recall
from sklearn.metrics import precision_score, recall_score
precision_score(testy,out,average='micro')

In [ ]:
recall_score(testy,out,average='micro')

In [ ]:
# To compute the F1 score, simply call the f1_score() function:
from sklearn.metrics import f1_score
f1_score(testy,out,average='micro')

Separate Stacking Model
We can now train a meta-learner that will best combine the predictions from the sub-models and ideally perform better than any single sub-model.

The first step is to load the saved models.

We can use the load_model() Keras function and create a Python list of loaded models.

Next, we can train our meta-learner. This requires two steps:

Prepare a training dataset for the meta-learner.
Use the prepared training dataset to fit a meta-learner model.
We will prepare a training dataset for the meta-learner by providing examples from the test set to each of the submodels and collecting the predictions. In this case, each model will output three predictions for each example for the probabilities that a given example belongs to each of the three classes. Therefore, the 1,000 examples in the test set will result in five arrays with the shape [3000, 3].

We can combine these arrays into a three-dimensional array with the shape [3000, 5, 3] by using the dstack() NumPy function that will stack each new set of predictions.

As input for a new model, we will require 1,000 examples with some number of features. Given that we have five models and each model makes three predictions per example, then we would have 15 (3 x 5) features for each example provided to the submodels. We can transform the [3000, 5, 3] shaped predictions from the sub-models into a [3000, 15] shaped array to be used to train a meta-learner using the reshape() NumPy function and flattening the final two dimensions. The stacked_dataset() function implements this step.

Once fit, we can use the stacked model, including the members and the meta-learner, to make predictions on new data.

This can be achieved by first using the sub-models to make an input dataset for the meta-learner, e.g. by calling the stacked_dataset() function, then making a prediction with the meta-learner. The stacked_prediction() function below implements this.

We can use this function to make a prediction on new data; in this case, we can demonstrate it by making predictions on the test set.

In [ ]:
'''
# stacked generalization with linear meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack


# load models from file
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = 'models/model_' + str(i + 1) + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models

all_models = list()
filename = '../input/finalized-model2/finalized_model.h5'
model = load_model(filename)
for i in range(5):
    all_models.append(model)

    
    
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
    stackX = None
    for model in members:
        # make prediction
        yhat = model.predict(inputX, verbose=0)
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = dstack((stackX, yhat))
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

Once prepared, we can use this input dataset along with the output, or y part, of the test set to train a new meta-learner.

In this case, we will train a simple logistic regression algorithm from the scikit-learn library.

Logistic regression only supports binary classification, although the implementation of logistic regression in scikit-learn in the LogisticRegression class supports multi-class classification (more than two classes) using a one-vs-rest scheme. The function fit_stacked_model() below will prepare the training dataset for the meta-learner by calling the stacked_dataset() function, then fit a logistic regression model that is then returned.

We can call this function and pass in the list of loaded models and the training dataset.

# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # fit standalone model
    model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    model.fit(stackedX, inputy)
    return model

# fit stacked model using the ensemble
model = fit_stacked_model(members, testX, testy)




# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

# evaluate model on test set
yhat = stacked_prediction(members, model, testX)
acc = accuracy_score(testy, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

'''